# Mouse Cortex Workflow

## Imports

In [ ]:
# Note we cap the pandas version as cytocipher caps the seaborn version
 #!pip install seaborn scanpy cytocipher pandas==1.5.3 numpy matplotlib ipywidgets scipy gseapy snapatac2 scikit-optimize tabulate

In [ ]:
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import scanpy as sc
import gseapy as gp
import seaborn as sns
import cytocipher as cc
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from scipy.stats import median_abs_deviation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import snapatac2 as snap

from skopt import gp_minimize
from skopt.space import Integer, Real
from sklearn.metrics import silhouette_score
from tabulate import tabulate
from scipy.stats import ttest_ind

In [ ]:
sc.settings.verbosity = 3  # log what scanpy does in the background
sns.set_theme(
    style="whitegrid",
    rc={"figure.figsize": (10, 8), "patch.edgecolor": "black"},
)  # make our plots bigger, and style them with seaborn

## Load Data

In [ ]:
# Load our merged gene expression count data
data = sc.read_h5ad("./gex.h5ad")
# See what data object looks like
data

## Pre-Processing (Quality Control)

In [ ]:
# Convert AnnData observation DataFrame to a pandas DataFrame
obs_df = data.obs

# Rename the 'condition' column to 'genotype'
obs_df.rename(columns={'condition': 'genotype'}, inplace=True)

# Assign the updated observation DataFrame back to AnnData object
data.obs = obs_df


In [ ]:
# Calculate QC metrics 
droplet_metrics, gene_metrics = sc.pp.calculate_qc_metrics(
    data, qc_vars=["mt"], inplace=False
)
droplet_metrics[["sample", "age", "genotype"]] = data.obs[["sample", "age", "genotype"]]
gene_metrics["mt"] = data.var["mt"]

In [ ]:
# Create summary table of stats BEFORE pre-processing (but after AMULET and CellBender)
# Change "sample" to "age" or "condition" if you want to group another way

pre_qc_summary = (
    droplet_metrics.assign(droplets=1)
    .groupby("sample")
    .agg(
        {
            "droplets": "sum",
            "n_genes_by_counts": np.median,
            "total_counts": np.median,
            "pct_counts_mt": np.median,
        }
    )
    .rename(
        {
            "n_genes_by_counts": "median_genes",
            "total_counts": "median_transcripts",
            "pct_counts_mt": "median_pct_transcripts_mt",
        },
        axis="columns",
    )
)
pre_qc_summary

When plotting with `"sample"` grouping, we may note:
- `E18_Con_1` has a massively larger droplet count.
- `E18_Con_1` has a much lower number of unique genes (less than half that of all others).
- `E18_Con_1` has a much lower number of transcripts (about half that of the next lowest).
- `E18_Con_2` has an unnusually high mitochondrial percentage.
- `E14` has lower mitochondrial count percentage compared to all `E18` samples.

In [ ]:
sns.jointplot(
    data=droplet_metrics,
    x="log1p_total_counts",
    y="log1p_n_genes_by_counts",
    kind="hex",
)

We may note a skew to lower gene counts, and a cluster of droplets with absolutely no transcripts/genes.
There is a slight arc to the hex plot.

In [ ]:
# Plot out our QC metrics
group_by = (
    "sample"  # change to "sample", "condition", or "age" to group, None to plot summary
)
for feature in ["n_genes_by_counts", "total_counts", "pct_counts_mt"]:
    sns.violinplot(data=droplet_metrics, y=feature, x=group_by)
    sns.stripplot(
        data=droplet_metrics, y=feature, x=group_by, s=0.5, color="black", jitter=0.4
    )
    plt.show()

When grouping by `"source"`, we may note that `E18_Con_1` has a very skewed distribution compared to the other samples, and an unusually long `pct_counts_mt` tail.

In [ ]:
sns.jointplot(
    data=droplet_metrics,
    x="log1p_total_counts",
    y="log1p_n_genes_by_counts",
    hue="sample",
)

If we split the prior plot by source, we may observe `E18_Con_1` is responsible for much of the skew.

In [ ]:
# We now use median absolution deviations to identify outlier droplets using our QC metrics.
# We do this per sample, as the prior plots show `E18_Con_1` skews heavily and we want to avoid it influencing other samples.


def is_outlier(metric_values, nmads: int = 3):
    # Helper to identify outliers using median absolute deviations (MAD)
    return (
        metric_values
        < np.median(metric_values) - nmads * median_abs_deviation(metric_values)
    ) | (
        np.median(metric_values) + nmads * median_abs_deviation(metric_values)
        < metric_values
    )


# For each sample, mark droplets with:
# - transcript or gene counts more than 5 MAD away,
# - a percentage of mitochondrial counts more than 3 MAD away or over 8%.
droplet_metrics["outlier"] = (
    droplet_metrics.groupby("sample")
    .apply(
        lambda x: (
            is_outlier(x["log1p_total_counts"], 5)
            | is_outlier(x["log1p_n_genes_by_counts"], 5)
            | is_outlier(x["pct_counts_mt"], 3)
            | (x["pct_counts_mt"] > 8)
        )
    )
    .reset_index(level=0, drop=True)
)

sns.jointplot(
    data=droplet_metrics,
    x="log1p_total_counts",
    y="log1p_n_genes_by_counts",
    hue="outlier",
)

This appears to truncate the tails relatively successfully, though we note a slight skew remains in the gene counts.

In [ ]:
sns.jointplot(
    data=droplet_metrics[~droplet_metrics["outlier"]],
    x="log1p_total_counts",
    y="log1p_n_genes_by_counts",
    hue="sample",
)

We may verify this is skew is due to `E18_Con_1`.

In [ ]:
# Now we recalculate the summary table with our outliers excluded
post_qc_summary = (
    droplet_metrics[~droplet_metrics["outlier"]]
    .assign(droplets=1)
    .groupby("sample")
    .agg(
        {
            "droplets": "sum",
            "n_genes_by_counts": np.median,
            "total_counts": np.median,
            "pct_counts_mt": np.median,
        }
    )
    .rename(
        {
            "n_genes_by_counts": "median_genes",
            "total_counts": "median_transcripts",
            "pct_counts_mt": "median_pct_transcripts_mt",
        },
        axis="columns",
    )
)
post_qc_summary

In [ ]:
# And use this to find the overall change
post_qc_summary - pre_qc_summary

We can see if we do apply this outlier filtering, `E18_Con_1` improves, but not sufficiently.
We may note that as we use MAD for outlier detection per sample, the skew in `E18_Con_1` will thus extend the MAD and so it has weaker thresholds.

In [ ]:
# We now exclude the outlier droplets, plus mitochondrial genes.
# It is likely that including `E18_Con_1` will skew further processing, so we also exclude that.
# WARN: Should we be dropping mitochondrial genes?
data = data[
    ~droplet_metrics["outlier"] & (data.obs["sample"] != "E18_Con_1"), ~data.var["mt"]
].copy()
data

In [ ]:
# Normalise transcript counts to median, to correct for varied library size.
data.layers["norm_10k"] = data.X.copy()
sc.pp.normalize_total(data, target_sum=1e4, layer="norm_10k")
sc.pp.normalize_total(data)

In [ ]:
sc.pp.log1p(data)
sc.pp.log1p(data, layer="raw")

## Exploration

In [ ]:
# Plot the highest expressed genes per sample
for sample_tag in data.obs["sample"].unique():
    subset_data = data[data.obs["sample"] == sample_tag]
    sc.pl.highest_expr_genes(subset_data, n_top=20, show=False).set(title=sample_tag)

In [ ]:
# Plot the highest expressed genes per genotype
for sample_tag in data.obs["genotype"].unique():
    subset_data = data[data.obs["genotype"] == sample_tag]
    sc.pl.highest_expr_genes(subset_data, n_top=20, show=False).set(title=sample_tag)

## Clustering

In [ ]:
# Here we just tune `min_mean` to ensure 0-mean-expression genes aren't included
sc.pp.highly_variable_genes(data, min_mean=0.0125)
# sc.pp.highly_variable_genes(data, min_mean=0.1, max_mean=0.8, min_disp=1) - this looks like the seurat details?
# Why do we have some negative dispersion?
sc.pl.highly_variable_genes(data)

In [ ]:
# We expect about 2000-3000 genes
data.var.highly_variable.sum()

<div class="alert alert-block alert-danger">
The following section on PCA and MP is not included in the final workflow, as we opted to use spectral dimensionality reduction instead.
</div>

We now manually perform PCA:
1. We use the subset of highly variable genes, matching the `scanpy` PCA implementation;
2. We scale the expression matrix to unit variance and zero mean per gene, which ensures equal contribution to principle components;
3. We densify the subset matrix.

In [ ]:
# scaled_subset_gex = StandardScaler().fit_transform(
#     data.X[:, data.var["highly_variable"]].T.A
# ).T

# # Is this consistent to Seurat?

In [ ]:
# # Use principle component analysis for dimensionality reduction (only first 100 components)
# pca = PCA(svd_solver="arpack", n_components=100)
# pca.fit(scaled_subset_gex)

# fig, ax = plt.subplots()
# ax.plot(np.log(pca.explained_variance_ratio_))
# ax.set_ylabel("Explained Variance")
# ax.set_xlabel("Principle Component")

# plt.show()

Traditional PCA would now select the top-N principle components to truncate to, however this relies on manual interpretation of the above plot.  
Specifically, we would need to locate the "elbow" (which falls between 30-50 for the above plot, as it's in log-scale).

We may better select this threshold by examining the distribution of eigenvalues in a permutation test per Marchenko Pastur law:
1. We calculate the explained variance (eigenvalues) of our data.
2. We recalculate the eigenvalues for many permutations of the data.
3. We estimate the distributions of our eigenvalues to find the threshold in significance.

Specifically, we may calculate the upper and lower bounds corresponding to the null distribution of eigenvalues:
$$
\lambda_{\pm} = \sigma^2 {(1\pm\sqrt{\mu})}^2 = {(1\pm\sqrt{\mu})}^2
$$
and then the PDF of eigenvalues:
$$
\text{PDF}(\lambda) = \frac{1}{2\pi\sigma^2} \frac{\sqrt{(\lambda_+-\lambda)(\lambda-\lambda_-)}}{\lambda\mu} = \frac{1}{2\pi} \frac{\sqrt{(\lambda_+-\lambda)(\lambda-\lambda_-)}}{\lambda\mu}
$$
where $\mu$ represents the ratio of matrix shape, and $\sigma$ is the variance (unit hence the simplified forms above).

In [ ]:
# # This process is offloaded to HPC, as we need ~1000 permutations.
# np.save("./gex_matrix.npy", scaled_subset_gex)

In [ ]:
# Now we load the sampled eigenvalues (from the permutation code).
#bg_evs = np.load("sampled_evs.npy")

In [ ]:
# # We calculate the theoretical upper and lower bounds for eigenvalues of the null per MP law.
# num_droplets, num_genes = scaled_subset_gex.shape
# mu = num_genes / num_droplets
# mp_upper = (1 + np.sqrt(mu)) ** 2
# mp_lower = (1 - np.sqrt(mu)) ** 2
# mp_lower, mp_upper

In [ ]:
# # We may plot this PDF
# mp_xs = np.linspace(mp_lower, mp_upper, 1000)
# mp_ys = np.sqrt((mp_upper - mp_xs)*(mp_xs - mp_lower)) / (2 * np.pi * mp_xs * mu)

# fig, ax = plt.subplots(figsize=(12,4))
# ax.plot(mp_xs, mp_ys)

# for ev in pca.explained_variance_:
#     if ev > mp_upper:
#         ax.arrow(
#             ev,
#             0.2,
#             0,
#             -0.2,
#             width=0.1,
#             head_width=1,
#             head_length=0.1,
#             length_includes_head=True,
#             color="red",
#         )

# plt.show()

In [ ]:
# fig, ax = plt.subplots()
# ax.plot(bg_evs.T, color="blue", alpha=0.01)
# ax.plot(pca.explained_variance_, color="black", linestyle="dashed")
# ax.plot(mp_xs, mp_ys)
# ax.legend(
#     [
#         Line2D([], [], color="blue"),
#         Line2D([], [], color="black", linestyle="dashed")
#     ],
#     ["samples", "truth"],
# )
# plt.show()

In [ ]:
# optimal_pcs = 50 # run PCA permutation test on Bunya and set this as that result.

In [ ]:
# sc.tl.pca(data, n_comps=optimal_pcs)
# sc.pl.pca(data)

It is worth noting that the application of PCA in single-cell analysis (and biology in general) is under some scrutiny.
This type of data violates many of the assumptions of PCA - see [10.1073/pnas.2311420120](https://www.pnas.org/doi/10.1073/pnas.2311420120).
We observe such an artifact in the curve of data across PC1 and PC2.

We did attempt constructing a neighbourhood graph on the raw data, but this gave poor clustering and blobby projections in UMAP.
As such, we instead use PCA but utilise the aforementioned permutation test to attempt to achieve a balance in approach.

In [ ]:
# sc.pp.neighbors(data, n_pcs=optimal_pcs)
# sc.tl.umap(data)

<div class="alert alert-block alert-danger">
Here concludes the PCA section - we should now include the spectral dimensionality reduction etc
</div>

## Spectral analysis

### DR on HVG and inital over-clustering visualisation

In [ ]:
data.obsm.keys()

In [ ]:
snap.tl.spectral(data, features=data.var.highly_variable)

In [ ]:
snap.tl.umap(data)

In [ ]:
# # Optimisation of leiden resolution and K for KNN graph

# #K = 81, leiden resolution = 0.85, n_genes = 14, enrichment method of "code" --> these values are associated with cytocipher, which is now deleted. 

# expected_clusters = 22 # Number of expected biological cell-types/clusters based off of those present in literature 

# def objective_function(params):
#     k, resolution_parameter = params

#     # Preprocessing: Update based on your dataset's needs
#     sc.pp.neighbors(data, n_neighbors=k, use_rep='X_spectral', metric='cosine')
#     sc.tl.leiden(data, resolution=resolution_parameter, n_iterations=-1) #flavor='igraph' 

#     # Compute silhouette score: Adjust 'metric' as needed
#     labels = data.obs['leiden'].astype(int)

#     silhouette = silhouette_score(data.obsm['X_spectral'], labels, metric='cosine')

#     # Calculate deviation from expected clusters
#     n_clusters = len(np.unique(labels))
#     cluster_deviation_penalty = abs(n_clusters - expected_clusters) / expected_clusters

#     objective_value = silhouette - cluster_deviation_penalty  # Higher silhouette is better, penalties for deviation

#     return -objective_value  # Minimize this value, hence the negative sign


# # Ranges for each param
# space = [
#     Integer(50, 120, name='k'), 
#     Real(0.4, 2.0, name='resolution_parameter'), 
# ]

# # Run the Bayesian optimizer
# result = gp_minimize(objective_function, space, n_calls=20, acq_func="EI", random_state=0)

# # Extract the best parameters
# best_k, best_resolution_parameter = result.x

# print(f"Optimal k: {best_k}, Optimal Resolution: {best_resolution_parameter}")

# #Optimal k: 91, Optimal Resolution: 1.750825197729628

In [ ]:
# Use outputted values from the iterative cluster refinement: 
# Use these values: Optimal k: 104, Optimal Resolution: 0.6547701310692625
#snap.pp.knn(data, n_neighbors=91)
sc.pp.neighbors(data, n_neighbors=104, use_rep='X_spectral', metric='cosine')

In [ ]:
sc.tl.leiden(data, resolution=0.6547701310692625)

In [ ]:
sc.tl.umap(data)

In [ ]:
# Save the leiden clustering + umap 
data.write("clustered_gex.h5ad")

### Re-load pre-processed & clustered data

In [ ]:
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import scanpy as sc
import gseapy as gp
import seaborn as sns
import cytocipher as cc
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from scipy.stats import median_abs_deviation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import snapatac2 as snap

from skopt import gp_minimize
from skopt.space import Integer, Real
from sklearn.metrics import silhouette_score
from tabulate import tabulate
#import snapatac2 as snap

In [ ]:
# Load our merged gene expression count data that has been pre-processed and clustered 
# Run from this cell from now on 
data = sc.read_h5ad("clustered_gex.h5ad")

In [ ]:
# Convert AnnData observation DataFrame to a pandas DataFrame
obs_df = data.obs

# Rename the 'condition' column to 'genotype'
obs_df.rename(columns={'condition': 'genotype'}, inplace=True)

# Assign the updated observation DataFrame back to AnnData object
data.obs = obs_df


In [ ]:
# Plot UMAP with custom colormaps for conditions - scanpy 
sc.pl.umap(data, color=["leiden",'sample', 'age', 'genotype'],ncols=2, wspace=0.4)

In [ ]:
# # Plot UMAP with custom colormaps for conditions - snapatac
# for color in ["sample", "leiden","age", "genotype"]:
#     snap.pl.umap(data, color=color, width=800, height=650, scale=0.5)

### Differential expression analysis

In [ ]:
#Perform differential expression analysis on CON VS. HOM
sc.tl.rank_genes_groups(data, groupby='genotype', method='wilcoxon')

In [ ]:
# Retrieve the differential expression analysis result
result = data.uns['rank_genes_groups']

# Extract the group names for Con and Hom conditions from data.obs
con_condition = 'Con'
hom_condition = 'Hom'

# Filter genes for Con condition with p-value > 0.05 and showing log fold change
con_filtered_genes = pd.DataFrame({
    'Gene': result['names'][f'{con_condition}'][:20],  # Top 10 genes
    'p-value': result['pvals'][f'{con_condition}'][:20],
    'logFC': result['logfoldchanges'][f'{con_condition}'][:20]
})

# Filter genes for Hom condition with p-value > 0.05 and showing log fold change
hom_filtered_genes = pd.DataFrame({
    'Gene': result['names'][f'{hom_condition}'][:20],  # Top 10 genes
    'p-value': result['pvals'][f'{hom_condition}'][:20],
    'logFC': result['logfoldchanges'][f'{hom_condition}'][:20]
})

# Display the top 10 differentially expressed genes for Con condition
print("Top 10 differentially expressed genes for Con condition:")
print(con_filtered_genes)

# Display the top 10 differentially expressed genes for Hom condition
print("\nTop 10 differentially expressed genes for Hom condition:")
print(hom_filtered_genes)

In [ ]:
# Filter genes for Con condition with p-value > 0.05 and showing log fold change
con_filtered_genes = pd.DataFrame({
    'Gene': result['names'][con_condition][:10],  # Top 10 genes
    'p-value': result['pvals'][con_condition][:10],
    'logFC': result['logfoldchanges'][con_condition][:10]
})

# Filter genes for Hom condition with p-value > 0.05 and showing log fold change
hom_filtered_genes = pd.DataFrame({
    'Gene': result['names'][hom_condition][:10],  # Top 10 genes
    'p-value': result['pvals'][hom_condition][:10],
    'logFC': result['logfoldchanges'][hom_condition][:10]
})

# Filter genes with p-values greater than 0.05
con_filtered_genes = con_filtered_genes[con_filtered_genes['p-value'] > 0.05]
hom_filtered_genes = hom_filtered_genes[hom_filtered_genes['p-value'] > 0.05]

# Display the top 10 differentially expressed genes for Con condition
print("Top 10 differentially expressed genes for Con condition:")
print(con_filtered_genes)

# Display the top 10 differentially expressed genes for Hom condition
print("\nTop 10 differentially expressed genes for Hom condition:")
print(hom_filtered_genes)


In [ ]:
# Create separate volcano plots for Con and Hom conditions
for condition, filtered_genes in zip(['Con', 'Hom'], [con_filtered_genes, hom_filtered_genes]):
    plt.figure(figsize=(8, 6))
    plt.scatter(filtered_genes['logFC'], -np.log10(filtered_genes['p-value']), color='blue', alpha=0.5)

    # Add labels for significant genes
    for i, gene in enumerate(filtered_genes['Gene']):
        plt.text(filtered_genes['logFC'].iloc[i], -np.log10(filtered_genes['p-value']).iloc[i], gene, fontsize=8, ha='center', va='bottom')

    plt.xlabel('Log Fold Change')
    plt.ylabel('-log10(p-value)')
    plt.title(f'Volcano Plot of Differentially Expressed Genes ({condition})')
    plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
    plt.axhline(y=-np.log10(0.05), color='red', linestyle='--', linewidth=1)
    plt.grid(True)
    plt.show()


In [ ]:
# Convert top 10 differentially expressed genes for Con condition to table format
con_table = tabulate(con_filtered_genes, headers='keys', tablefmt='pretty')

# Convert top 10 differentially expressed genes for Hom condition to table format
hom_table = tabulate(hom_filtered_genes, headers='keys', tablefmt='pretty')

# Display the tables
print("Top 10 differentially expressed genes for Con condition:")
print(con_table)

print("\nTop 10 differentially expressed genes for Hom condition:")
print(hom_table)


In [ ]:
# Step 3: Filter significant genes and calculate fold changes
result = data.uns['rank_genes_groups']

groups = result['names'].dtype.names
diff_genes = pd.DataFrame({group + '_' + key[:1]: result[key][group]
for group in groups for key in ['names', 'pvals_adj', 'logfoldchanges']})


In [ ]:
diff_genes = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals_adj', 'logfoldchanges']})


In [ ]:
# Step 4: Statistical significance testing
# Adjusted p-values
adjusted_p_values_con = diff_genes['Con_p']
adjusted_p_values_hom = diff_genes['Hom_p']


In [ ]:
# Filter genes with adjusted p-values less than 0.05
significant_genes_con = diff_genes[diff_genes['Con_p'] < 0.05]
significant_genes_hom = diff_genes[diff_genes['Hom_p'] < 0.05]


In [ ]:
sc.pl.rank_genes_groups_violin(data, 'genotype', n_genes=8)

In [ ]:
result = data.uns['rank_genes_groups']
groups = result['names'].dtype.names
filtered_data = {
    group + '_' + key[:1]: result[key][group]
    for group in groups
    for key in ['names', 'pvals', 'logfoldchanges']
    if key == 'pvals' and any(p_val < 0.05 for p_val in result[key][group])
}
pd.DataFrame(filtered_data).head(5)


In [ ]:
sc.pl.rank_genes_groups(data, figsize=(6, 4), show=False)

In [ ]:
# Combine age and condition information into a single column
combined_column = data.obs['age'].astype(str) + '_' + data.obs['condition'].astype(str)
data.obs['age_condition'] = combined_column

# Subset data for E14 and E18 separately
data_E14 = data[data.obs['age'] == 'E14']
data_E18 = data[data.obs['age'] == 'E18']

for sample_tag in data.obs["age_condition"].unique():
        subset_data = data[data.obs["age_condition"] == sample_tag]

data_E14_Hom = data[data.obs['age_condition'] == 'E14_Hom_'

# Plot UMAP for E14 samples
sc.pl.umap(data_E14, color=['genotype', 'Pax6'], ncols=3)

# Plot UMAP for E18 samples
sc.pl.umap(data_E18, color=['age_condition', 'Pax6'], ncols=3)


In [ ]:
snap.pl.umap(subset_data, color="Pax6", width=800, height=650, scale=0.5)

## Feature plots (for marker genes)

In [ ]:
# # Canonical marker genes compiled from literature 
genes_of_interest = ['Pax6']
for gene in genes_of_interest:
    for (age_tag, genotype_tag), subset_df in data.obs.groupby(["age", "genotype"]):
        subset_data = data[subset_df.index, :]
        sc.pl.umap(subset_data, color=gene, title=f'{age_tag} {genotype_tag} - {gene} Expression', ncols=2, wspace=0.1 )    
          

#sc.pl.umap(subset_data, color=['age','condition', 'sample', 'Eomes', 'Mki67', 'Gad2', 'Reln', 'Nsd1', 'Pax6', 'Bcl11b', 'Satb2', 'Sox5', 'Hbb-bs'], ncols=5)
          

In [ ]:
# stats for the expression level of a given gene for a given genotype

# count column - it's from calling .describe and is just counting the number of values thus the number of droplets in total

genes = ["Pax6", "Eomes", "Mki67", "Bcl11b"]

expression_df = (
    data.obs[["age", "genotype"]]
    .join(data[:, genes].to_df())
    .melt(
        id_vars=["age", "genotype"],
        value_vars=genes,
        var_name="gene",
        value_name="expression",
    )
    .assign(expressed=lambda x: x["expression"] > 0)
)

sns.violinplot(
    data=expression_df,
    x="gene",
    y="expression",
    hue="genotype",
    split=True,
    inner="quart",
).set(xlabel="Gene", ylabel="Expression", title="Gene Expression by Genotype")

grouped_expression_df = expression_df.groupby(["gene", "genotype"])
grouped_expression_df.describe().assign(
    proportion_of_droplets_expressing=(grouped_expression_df["expressed"].mean())
)


In [ ]:
# Perform Welch's t-test for each gene across genotypes

gene_t_tests = {}
for gene in genes:
    for genotype in data.obs["genotype"].unique():
        # Select data for the current gene and genotype
        gene_data = expression_df[(expression_df["gene"] == gene) & (expression_df["genotype"] == genotype)]
        
        # Get the proportion of cells expressing the gene for the current genotype
        proportion_expressed = gene_data["expressed"].mean()
        
        # Store the proportion in the gene_t_tests dictionary
        gene_t_tests[(gene, genotype)] = proportion_expressed

# Print t-test results
for gene in genes:
    genotype_1 = data.obs["genotype"].unique()[0]
    genotype_2 = data.obs["genotype"].unique()[1]
    proportion_1 = gene_t_tests[(gene, genotype_1)]
    proportion_2 = gene_t_tests[(gene, genotype_2)]
    
    t_stat, p_value = ttest_ind(expression_df[expression_df["gene"] == gene]
                                [expression_df["genotype"] == genotype_1]["expressed"],
                                expression_df[expression_df["gene"] == gene]
                                [expression_df["genotype"] == genotype_2]["expressed"],
                                equal_var=False)
    
    print(f"Gene: {gene}")
    print(f"Proportion expressed in {genotype_1}: {proportion_1}")
    print(f"Proportion expressed in {genotype_2}: {proportion_2}")
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")
    if p_value < 0.05:
        print("Significant difference in expression between genotypes")
    else:
        print("No significant difference in expression between genotypes")
    print()


## Azimuth label transfer

In [ ]:
# Look at the columns
# There are separate columns for each of the 16 datasets - there's a column for the label and then a column for the score of that label.
data.obs.keys()

In [ ]:
print([x for x in data.obs.keys() if x.startswith("label_") and not x.endswith("_score")])

# List of Azimuth label transfer references 
all_references = ['label_loo_2019_E14.5', 'label_loo_2019_P0', 'label_dibella_2021_E13.5', 'label_zhang_2021_E10.5', 'label_dibella_2021_E14.5', 'label_dibella_2021_E15.5', 'label_dibella_2021_E10.5', 'label_zhang_2021_E16.5', 'label_zhang_2021_E18.5', 'label_zhang_2021_E12.5', 'label_zhang_2021_E14.5', 'label_dibella_2021_E11.5', 'label_dibella_2021_E12.5', 'label_zhang_2021_E15.5', 'label_dibella_2021_E17.5', 'label_dibella_2021_E16.5', 'label_dibella_2021_E18.5b', 'label_dibella_2021_E18.5a']

# List of relevant age references 
age_references =  ['label_dibella_2021_E13.5', 'label_zhang_2021_E14.5','label_loo_2019_E14.5','label_dibella_2021_E14.5', 'label_dibella_2021_E17.5', 'label_zhang_2021_E18.5', 'label_dibella_2021_E18.5b', 'label_dibella_2021_E18.5a']


In [ ]:
print([x for x in data.obs.keys() if x.startswith("label_") and x.endswith("_score")])

# List of Azimuth label transfer re
all_references_scores = ['label_loo_2019_E14.5_score', 'label_loo_2019_P0_score', 'label_dibella_2021_E13.5_score', 'label_zhang_2021_E10.5_score', 'label_dibella_2021_E14.5_score', 'label_dibella_2021_E15.5_score', 'label_dibella_2021_E10.5_score', 'label_zhang_2021_E16.5_score', 'label_zhang_2021_E18.5_score', 'label_zhang_2021_E12.5_score', 'label_zhang_2021_E14.5_score', 'label_dibella_2021_E11.5_score', 'label_dibella_2021_E12.5_score', 'label_zhang_2021_E15.5_score', 'label_dibella_2021_E17.5_score', 'label_dibella_2021_E16.5_score', 'label_dibella_2021_E18.5b_score', 'label_dibella_2021_E18.5a_score']

# List of relevant age references 
age_references_scores =  ['label_loo_2019_E14.5_score', 'label_loo_2019_P0_score', 'label_dibella_2021_E13.5_score', 'label_dibella_2021_E14.5_score', 'label_zhang_2021_E18.5_score','label_zhang_2021_E14.5_score','label_dibella_2021_E17.5_score', 'label_dibella_2021_E18.5b_score', 'label_dibella_2021_E18.5a_score']

In [ ]:
# # # For each reference, look at the transfer labelling to clusters and the Azimuth score of how well it worked, separated by E14 and E18 
# # # Are the scores lower for a particular cell-type? Is the reference poorly matched with low scores everywhere? 

# # # Transfer label UMAPs + Azimuth scores 

# for x in age_references:
#     for age_tag in data.obs["age"].unique():
#         subset_data = data[data.obs["age"] == age_tag]
#         filtered_scores = f"{x}_score"
#         filtered_data = subset_data[subset_data.obs[filtered_scores] > 0.5]
#         for color in [x, filtered_scores]:
#             print(age_tag)
#             snap.pl.umap(data, color=color, width=800, height=650, scale=0.5)

### Proportion of Azimuth labels to cell-types in clusters

In [ ]:
label_columns = [
    x
    for x in sorted(data.obs.columns)
    if x.startswith("label_") and not x.endswith("_score")
]
proportion_columns = ["age", "genotype", *label_columns]

In [ ]:
def label_fixer(label: str) -> str:
    if "[" in label:
        label = label.split("[")[0].strip()
    if label.startswith("Astrocytes (immature)"):
        label = "Astrocytes (immature)"
    if label.startswith("Endothelial"):
        label = "Endothelial cells"
    if label.startswith("Immature neuron"):
        label = "Neurons - Immature"
    if label == "Interneuron" or label.startswith("Int"):
        label = "Interneurons"
    if label == "Pericyte":
        label = "Pericytes"
    if label.startswith("RG"):
        label = "Radial glial cells"
    if label == "Migrating neurons" or (
        label.startswith("SVZ") and "migrating" in label
    ):
        label = "Neurons - Migrating"
    if label == "NP":
        label = "Apical progenitors"
    if label.startswith("Layer"):
        if not label.startswith("Layer "):
            label = f"Layer {label[5:]}"
        label = (
            label.replace("6b", "V-VI")
            .replace("4", "II-IV")
            .replace("V-VI-a", "V-VI")
            .replace("V-VI-b", "V-VI")
        )
        label = f"Neurons - {label}"
    return label


obs_df = data.obs[["leiden", *proportion_columns]].applymap(label_fixer)

In [ ]:
labels = np.unique(obs_df[proportion_columns].values)
palette = dict(zip(labels, sns.color_palette("Spectral", n_colors=len(labels))))

for cluster_idx, cluster_df in obs_df.groupby("leiden"):
    fig, ax = plt.subplots()
    ax.set_title(f"Cluster #{cluster_idx} Breakdown")

    for c in proportion_columns:
        proportions_dict = cluster_df[c].value_counts(normalize=True).to_dict()

        left = 0
        for label, proportion in proportions_dict.items():
            if proportion == 0:
                continue
            barcountainer = ax.barh(c, proportion, left=left, color=palette[label])

            if proportion > 0.1:
                ax.text(
                    x=barcountainer[0].get_x() + 0.01,
                    y=barcountainer[0].get_y()
                    + barcountainer[0].get_height() / 2
                    - 0.2,
                    s=f"{label} ({proportion*100:.0f}%)",
                    ha="left",
                    va="top",
                    size=8,
                )

            left += proportion

    ax.set_xticks([])
    ax.set_xlabel("Proportion")
    ax.grid(False)
    ax.invert_yaxis()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)

    plt.show()

### Manual annotation

In [ ]:
# Define a dictionary to map cell types to leiden clusters

cluster_cell_type_dict_combined = {
    '0': 'Neurons - immature',
    '1': 'Radial glial cells',
    '2': 'Neurons',
    '3': 'Radial glial cells',
    '4': 'Interneurons',
    '5': 'Neurons',
    '6': 'Neurons',
    '7': 'Interneurons',
    '8': 'Neurons - immature',
    '9': 'Neurons',
    '10': 'Intermediate progenitor cells',
    '11': 'Neurons',
    '12': 'Neurons',
    '13': 'Interneurons',
    '14': 'Neurons',
    '15': 'Astrocytes',
    '16': 'Neurons',
    '17': 'Pericytes',
    '18': 'Interneurons',
    '19': 'Cajal-Retzius cells',
    '20': 'Neurons',
    '21': 'Endothelial cells',
    '22': 'Microglia',
    '23': 'Oligodendrocytes',
}

In [ ]:
data.obs['cell_type'] = data.obs['leiden'].astype(str).map(cluster_cell_type_dict_combined)

In [ ]:
data.obs[['cell_type', 'leiden']]

In [ ]:
# Cell types per genotype (at both ages)


# for (age_tag, genotype_tag), subset_df in data.obs.groupby(["age", "genotype"]):
#     subset_data = data[subset_df.index, :]
#     sc.pl.umap(subset_data, color='cell_type', title=f'{age_tag} {genotype_tag} - Cell Types')

for (genotype_tag), subset_df in data.obs.groupby(["genotype"]):
    subset_data = data[subset_df.index, :]
    sc.pl.umap(subset_data, color='cell_type', title=f'{genotype_tag} - Cell Types')


In [ ]:
# Proportion of cell types within category, per genotype per age 

for (age_tag, genotype_tag), subset_df in data.obs.groupby(["age", "genotype"]):
    subset_data = data[subset_df.index, :].copy()
    sc.pl.umap(subset_data, color='cell_type', title=f'{age_tag} {genotype_tag} - Cell Types', ncols=2, wspace=0.1)
    print(
        subset_df["cell_type"].value_counts() / len(subset_df)
    )


In [ ]:
# from scipy.stats import ttest_ind

# # Dictionary to store t-test results for each cell type
# cell_type_t_tests = {}

# # Iterate over combinations of age and genotype
# for (age_tag, genotype_tag), subset_df in data.obs.groupby(["age", "genotype"]):
#     subset_data = data[subset_df.index, :].copy()
    
#     # Calculate proportion of each cell type within the subgroup
#     cell_type_proportions = subset_df["cell_type"].value_counts() / len(subset_df)
    
#     # Store proportions in the dictionary
#     for cell_type, proportion in cell_type_proportions.items():
#         if (age_tag, cell_type) not in cell_type_t_tests:
#             cell_type_t_tests[(age_tag, cell_type)] = {'Con': [], 'Hom': []}
#         cell_type_t_tests[(age_tag, cell_type)][genotype_tag].append(proportion)

# # Perform t-tests for each cell type
# for (age_tag, cell_type), proportions in cell_type_t_tests.items():
#     if len(proportions['Con']) > 0 and len(proportions['Hom']) > 0:  # Check if both genotypes have data
#         t_stat, p_value = ttest_ind(proportions['Con'], proportions['Hom'])
#         print(f"For {cell_type} at {age_tag}:")
#         print(f"T-statistic: {t_stat}")
#         print(f"P-value: {p_value}")
#         if p_value < 0.05:
#             print("Significant difference in proportions between genotypes")
#         else:
#             print("No significant difference in proportions between genotypes")
#         print()
#     else:
#         print(f"Not enough data for {cell_type} at {age_tag} to perform t-test")


In [ ]:
# Con and Hom on same plot 
for age_tag, subset_df in data.obs.groupby("age"):
    subset_data = data[subset_df.index, :].copy()
    subset_data.obs["cell_type_by_genotype"] = subset_df["genotype"].astype(str) + ":" + subset_df["cell_type"]

    sc.pl.umap(subset_data, color='cell_type_by_genotype', title=f'{age_tag} - Cell Types')
    print(
        subset_data.obs["cell_type_by_genotype"].value_counts() / len(subset_df)
    )

### Subset Layer 1 neurons across timepoints and genotypes

In [ ]:
results = []


for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    #print(subset_data_age.obs['age'].unique())
    for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
        subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
        subset_data_rgc = subset_data_genotype[subset_data_genotype.obs['cell_type'] == "Neurons", :].copy()
        results.append({'Age': age_tag, 'Genotype': genotype_tag, 'Count': len(subset_data_rgc)})

        
        print(f"Neurons in {age_tag} {genotype_tag} is {len(subset_data_rgc)}")
        
rgc_df = pd.DataFrame(results)




# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Create a bar plot
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='Age', y='Count', hue='Genotype', data=rgc_df, palette='pastel', ci=None)

# Adding labels and title
plt.xlabel('Age')
plt.ylabel('Count of Immature Neurons')
plt.title('Comparison of Immature Neuron Counts by Age and Genotype')
plt.legend(title='Genotype')

for p in bar_plot.patches:
    if p.get_height() > 0:  # Only annotate bars with height greater than 0
        bar_plot.annotate(format(int(p.get_height()), 'd'),  # Remove unnecessary decimals
                          (p.get_x() + p.get_width() / 2., p.get_height()),
                          ha = 'center', va = 'center',
                          size=10, xytext = (0, 8),
                          textcoords = 'offset points')
# Show the plot
plt.show()



In [ ]:
results = []

for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
        subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
        subset_data_cells = subset_data_genotype[subset_data_genotype.obs['cell_type'] == "Neurons", :].copy()
        
        # Count cells expressing the gene 'Bcl11b'
        bcl11b_count = subset_data_cells[:, data.var_names == 'Bcl11b'].X.sum()
        
        results.append({'Age': age_tag, 'Genotype': genotype_tag, 'Bcl11b_Count': bcl11b_count})
        
        print(f"Immature neurons expressing Bcl11b in {age_tag} {genotype_tag} is {bcl11b_count}")

# Convert results to DataFrame
bcl11b_df = pd.DataFrame(results)

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Create bar plot
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='Age', y='Bcl11b_Count', hue='Genotype', data=bcl11b_df, palette='pastel', ci=None)

# Adding labels and title
plt.xlabel('Age')
plt.ylabel('Count of Immature neurons expressing Bcl11b')
plt.title('Comparison of Immature neurons expressing Bcl11b by Age and Genotype')
plt.legend(title='Genotype')

# Annotate bars
for p in bar_plot.patches:
    if p.get_height() > 0:  # Only annotate bars with height greater than 0
        bar_plot.annotate(format(int(p.get_height()), 'd'),  # Remove unnecessary decimals
                          (p.get_x() + p.get_width() / 2., p.get_height()),
                          ha='center', va='center',
                          size=10, xytext=(0, 8),
                          textcoords='offset points')

# Show the plot
plt.show()


In [ ]:
# # UMAP OF BCL11B EXPRESSING CELLS (IMMATURE NEURONS)

for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[data.obs['age'] == age_tag, :].copy()
    for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
        subset_data_genotype = subset_data_age[subset_data_age.obs['genotype'] == genotype_tag, :].copy()
        
        # Define clusters expressing radial glial cells based on their cell type
        subset_data_genotype.obs['is_Neuron'] = subset_data_genotype.obs['cell_type'].str.startswith('Neurons')
        
        # Convert 'is_radial_glial' to category dtype
        subset_data_genotype.obs['is_Neuron'] = subset_data_genotype.obs['is_Neuron'].astype('category')
             
        # Plot the UMAP
        sc.pl.umap(subset_data_genotype, color=["is_Neuron"], title=f"{age_tag} {genotype_tag} Immature Neuron Expression")



# for age_tag, subset_age_df in data.obs.groupby(["age"]):
#     subset_data_age = data[data.obs['age'] == age_tag, :].copy()
#     for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
#         subset_data_genotype = subset_data_age[subset_data_age.obs['genotype'] == genotype_tag, :].copy()
        
#         # Define cells expressing Bcl11b based on their gene expression
#         is_bcl11b_expressing = subset_data_genotype[:, data.var_names == 'Bcl11b'].X.sum(axis=1) > 0
        
#         # Perform Leiden clustering on cells expressing Bcl11b
#         sc.tl.leiden(subset_data_genotype[is_bcl11b_expressing, :])
        
#         # Plot the UMAP with clusters expressing Bcl11b
#         sc.pl.umap(subset_data_genotype, color=["leiden", "Bcl11b"], title=f"{age_tag} {genotype_tag} Clusters and Bcl11b Expression")



In [ ]:
results = []

for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
        subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
        subset_data_cells = subset_data_genotype[subset_data_genotype.obs['cell_type'] == "Neurons", :].copy()
        
        # Count cells expressing the gene 'Fezf2'
        fezf2_count = subset_data_cells[:, data.var_names == 'Fezf2'].X.sum()
        
        results.append({'Age': age_tag, 'Genotype': genotype_tag, 'Fezf2_Count': fezf2_count})
        
        print(f"Cells expressing Fezf2 in {age_tag} {genotype_tag} is {fezf2_count}")

# Convert results to DataFrame
fezf2_df = pd.DataFrame(results)

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Create bar plot
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='Age', y='Fezf2_Count', hue='Genotype', data=fezf2_df, palette='pastel', ci=None)

# Adding labels and title
plt.xlabel('Age')
plt.ylabel('Count of Immature neurons expressing Fezf2')
plt.title('Comparison of Immature neurons expressing Fezf2 by Age and Genotype')
plt.legend(title='Genotype')

# Annotate bars
for p in bar_plot.patches:
    if p.get_height() > 0:  # Only annotate bars with height greater than 0
        bar_plot.annotate(format(int(p.get_height()), 'd'),  # Remove unnecessary decimals
                          (p.get_x() + p.get_width() / 2., p.get_height()),
                          ha='center', va='center',
                          size=10, xytext=(0, 8),
                          textcoords='offset points')

# Show the plot
plt.show()


In [ ]:
for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    #print(subset_data_age.obs['age'].unique())
    #subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
    subset_data_rgc = subset_data_age[subset_data_age.obs['cell_type'] == "Neurons - immature", :].copy()
    #print(subset_data_rgc.obs)
    sc.tl.rank_genes_groups(subset_data_rgc, groupby="genotype", method='wilcoxon')
    sc.pl.rank_genes_groups(subset_data_rgc, groupby="genotype", method='wilcoxon')
    sc.pl.rank_genes_groups_dotplot(
        subset_data_rgc,
        var_names = ["Bcl11b", 'Fezf2', "Sox5","Ldb2"],
        values_to_plot="logfoldchanges", cmap='bwr',
        vmin=-2,
        vmax=2,
        min_logfoldchange=3,
        colorbar_title='log fold change',
        title=f"{age_tag} Neuron \nMarker gene DEG Expression\nper genotype"
    )


#Threshold by p-value, get a set of top K up- and down-regulated genes

### Subset RGCs across timepoints and genotypes 

In [ ]:
results = []


for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    #print(subset_data_age.obs['age'].unique())
    for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
        subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
        subset_data_rgc = subset_data_genotype[subset_data_genotype.obs['cell_type'] == "Radial glial cells", :].copy()
        results.append({'Age': age_tag, 'Genotype': genotype_tag, 'Count': len(subset_data_rgc)})

        
        print(f"RGC in {age_tag} {genotype_tag} is {len(subset_data_rgc)}")
        
rgc_df = pd.DataFrame(results)




# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Create a bar plot
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(x='Age', y='Count', hue='Genotype', data=rgc_df, palette='pastel', ci=None)

# Adding labels and title
plt.xlabel('Age')
plt.ylabel('Count of Radial Glial Cells')
plt.title('Comparison of Radial Glial Cell Counts by Age and Genotype')
plt.legend(title='Genotype')

for p in bar_plot.patches:
    if p.get_height() > 0:  # Only annotate bars with height greater than 0
        bar_plot.annotate(format(int(p.get_height()), 'd'),  # Remove unnecessary decimals
                          (p.get_x() + p.get_width() / 2., p.get_height()),
                          ha = 'center', va = 'center',
                          size=10, xytext = (0, 8),
                          textcoords = 'offset points')
# Show the plot
plt.show()



#### Analysis of DEGs in RGCs

In [ ]:

for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    #print(subset_data_age.obs['age'].unique())
    #subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
    subset_data_rgc = subset_data_age[subset_data_age.obs['cell_type'] == "Radial glial cells", :].copy()
    #print(subset_data_rgc.obs)
    sc.tl.rank_genes_groups(subset_data_rgc, groupby="genotype", method='wilcoxon')
    sc.pl.rank_genes_groups(subset_data_rgc, groupby="genotype", method='wilcoxon')
    sc.pl.rank_genes_groups_dotplot(
        subset_data_rgc,
        var_names = [
    "Pax6",
    "Fabp7",
    "Vim",
    "Nes",
    "Sox2",
    "Hes1",
    "Hes5",
    "Ednrb",
    "Sptbn1",
    "Hmga2"
],
        values_to_plot="logfoldchanges", cmap='bwr',
        vmin=-0.5,
        vmax=0.5,
        min_logfoldchange=3,
        colorbar_title='log fold change', 
        title=f"{age_tag} Radial Glial Cell \nMarker gene DEG Expression\nper genotype", 
    )
    sc.pl.rank_genes_groups_dotplot(
        subset_data_rgc,
        var_names = [subset_data_rgc
],
        values_to_plot="logfoldchanges", cmap='bwr',
        vmin=-0.5,
        vmax=0.5,
        min_logfoldchange=3,
        colorbar_title='log fold change', 
        title=f"{age_tag} Radial Glial Cell \nMarker gene DEG Expression\nper genotype", 
    )


In [ ]:
# RGC UMAP 

# for age_tag, subset_age_df in data.obs.groupby(["age"]):
#     subset_data_age = data[subset_age_df.index, :].copy()
#     for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
#         subset_data_genotype.obs['cell_type'] = subset_data_genotype.obs['cell_type'].astype('category')
#         subset_data_genotype = subset_data_age[subset_genotype_df.index, :].copy()
#         subset_data_genotype.obs["is_glial"] = subset_data_genotype.obs['cell_type'].str.startswith('Radial')
#         # Convert 'cell_type' column to categorical dtype if it's not already
#         # subset_data_genotype.obs['cell_type'] = subset_data_genotype.obs['cell_type'].astype('category')

#         sc.pl.umap(subset_data_genotype, color=["is_glial"], title=f"{age_tag} {genotype_tag} Radial Glial Cell Expression")



for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[data.obs['age'] == age_tag, :].copy()
    for genotype_tag, subset_genotype_df in subset_data_age.obs.groupby(['genotype']):
        subset_data_genotype = subset_data_age[subset_data_age.obs['genotype'] == genotype_tag, :].copy()
        
        # Define clusters expressing radial glial cells based on their cell type
        subset_data_genotype.obs['is_radial_glial'] = subset_data_genotype.obs['cell_type'].str.startswith('Radial')
        
        # Convert 'is_radial_glial' to category dtype
        subset_data_genotype.obs['is_radial_glial'] = subset_data_genotype.obs['is_radial_glial'].astype('category')
             
        # Plot the UMAP
        sc.pl.umap(subset_data_genotype, color=["is_radial_glial"], title=f"{age_tag} {genotype_tag} Radial Glial Cell Expression")


In [ ]:
proportion_columns = ["genotype"]

In [ ]:
# Assuming 'data' and 'proportion_columns' are defined elsewhere
labels = np.unique(obs_df[proportion_columns].values)
palette = dict(zip(labels, sns.color_palette("Spectral", n_colors=len(labels))))

# Create a single figure and axis
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figure size as needed

# Variables to adjust the placement of each cluster's bars in the plot
group_gap = 0.5  # Gap between groups
bar_height = 0.4  # Height of each bar
current_bottom = 0  # Keeps track of where to start the next cluster

# Loop through each cluster
for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    for cluster_idx, cluster_df in enumerate(subset_data_age.obs.groupby("cell_type")):
        if cluster_idx > 0:
            current_bottom += group_gap  # Add a gap between groups
    
        # Add a title for each cluster within the plot
        ax.text(x=-0.01, y=current_bottom, s=f"{cluster_df[0]}", ha='right', va='center', weight='bold', fontsize=12)
    
        for idx, c in enumerate(proportion_columns):
            proportions_dict = cluster_df[1][c].value_counts(normalize=True).to_dict()
            left = 0
    
            # Commented out the display of subgroup names to the left of the bars
            # ax.text(x=-0.01, y=current_bottom + idx*(bar_height + 0.1), s=c, ha='right', va='center')
    
            for label, proportion in sorted(proportions_dict.items(), key=lambda x: x[1], reverse=True):
                if proportion == 0:
                    continue
    
                bar_container = ax.barh(y=current_bottom + idx*(bar_height + 0.1), width=proportion, height=bar_height, left=left, color=palette[label])
    
                # Adjust text position based on proportion to avoid crowding
                if proportion > 0.1:
                    ax.text(
                        x=left + proportion / 2,
                        y=current_bottom + idx*(bar_height + 0.1),
                        s=f"{label} ({proportion*100:.0f}%)",
                        ha="center",
                        va="center",
                        size=8,
                    )
    
                left += proportion
    
        current_bottom += (len(proportion_columns) * (bar_height + 0.1))  # Update the bottom for the next group

    # Finalize the plot
    ax.set_yticks([])
    ax.set_xlabel("Proportion of cell-type composed by genotype")
    ax.grid(False)
    ax.invert_yaxis()  # Optional: depends on how you want to display the groups
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    
    
    
    
    
    plt.show()

## DEG Enrichment analysis EnrichR

In [ ]:
sc.tl.rank_genes_groups(data,
                        groupby='genotype',
                        use_raw=False,
                        method='wilcoxon',
                        groups=["Hom"],
                        key=["Genotypes"], 
                        reference='Con',
                        pval_cutoff=0.05)

#sc.tl.rank_genes_groups(data, groupby='spectral_leiden_merged', method = 'wilcoxon', key_added ="spectral", pval_cutoff=0.05)



In [ ]:
sc.pl.rank_genes_groups(data, n_genes=25, sharey=False)

In [ ]:
# get deg result
result = data.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})


In [ ]:
degs.head()

In [ ]:
degs.shape

In [ ]:
# subset up or down regulated genes
degs_sig = degs[degs.Hom_pvals_adj < 0.05]
degs_up = degs_sig[degs_sig.Hom_logfoldchanges > 0]
degs_dw = degs_sig[degs_sig.Hom_logfoldchanges < 0]

In [ ]:
degs_up.shape

In [ ]:
degs_dw.shape

### GSEAPY

In [ ]:
#see full list of latest enrichr library names, which will pass to -g parameter:
names = gp.get_library_name()

# show top 20 entries.
print(names[:20])

In [ ]:
#GO_Biological_Process_2021

# Enricr API
enr_up = gp.enrichr(degs_up.Hom_names,
                    gene_sets='Mouse_Gene_Atlas',
                    outdir=None)

In [ ]:

# trim (go:...)
enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]

# dotplot
gp.dotplot(enr_up.res2d, figsize=(3,5), title="Up", cmap = plt.cm.autumn_r)
plt.show()

enr_dw = gp.enrichr(degs_dw.Hom_names,
                    gene_sets='Mouse_Gene_Atlas',
                    outdir=None)


enr_dw.res2d.Term = enr_dw.res2d.Term.str.split(" \(GO").str[0]
gp.dotplot(enr_dw.res2d,
           figsize=(3,5),
           title="Down",
           cmap = plt.cm.winter_r,
           size=5)
plt.show()

# concat results
enr_up.res2d['UP_DW'] = "UP"


enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(), enr_dw.res2d.head()])

from gseapy.scipalette import SciPalette
sci = SciPalette()
NbDr = sci.create_colormap()
# NbDr

# display multi-datasets
ax = gp.dotplot(enr_res,figsize=(3,5),
                x='UP_DW',
                x_order = ["UP","DOWN"],
                title="GO_BP",
                cmap = NbDr.reversed(),
                size=3,
                show_ring=True)
ax.set_xlabel("")
plt.show()

ax = gp.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="GO_BP",
                color = ['b','r'])

In [ ]:
# Subset to RGCs

for age_tag, subset_age_df in data.obs.groupby(["age"]):
    subset_data_age = data[subset_age_df.index, :].copy()
    subset_data_rgc = subset_data_age[subset_data_age.obs['cell_type'] == "Radial glial cells", :].copy()
    sc.tl.rank_genes_groups(subset_data_rgc, groupby="genotype", use_raw=False, method='wilcoxon', key=['RGC_Genotypes'],reference='Con', pval_cutoff=0.05)

In [ ]:
# get deg result
result = data.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

degs.head()

In [ ]:
# subset up or down regulated genes
degs_sig = degs[degs.Hom_pvals_adj < 0.05]
rgc_degs_up = degs_sig[degs_sig.Hom_logfoldchanges > 0]
rgc_degs_dw = degs_sig[degs_sig.Hom_logfoldchanges < 0]

In [ ]:
rgc_degs_up.shape

In [ ]:
rgc_degs_dw.shape

In [ ]:
#GO_Biological_Process_2021

# Enricr API
enr_up = gp.enrichr(rgc_degs_up.Hom_names,
                    gene_sets='Mouse_Gene_Atlas',
                    outdir=None)

In [ ]:

# trim (go:...)
enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]

# dotplot
gp.dotplot(enr_up.res2d, figsize=(3,5), title="Up", cmap = plt.cm.autumn_r)
plt.show()

enr_dw = gp.enrichr(rgc_degs_dw.Hom_names,
                    gene_sets='Mouse_Gene_Atlas',
                    outdir=None)


enr_dw.res2d.Term = enr_dw.res2d.Term.str.split(" \(GO").str[0]
gp.dotplot(enr_dw.res2d,
           figsize=(3,5),
           title="Down",
           cmap = plt.cm.winter_r,
           size=5)
plt.show()

# concat results
enr_up.res2d['UP_DW'] = "UP"


enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(), enr_dw.res2d.head()])

from gseapy.scipalette import SciPalette
sci = SciPalette()
NbDr = sci.create_colormap()
# NbDr

# display multi-datasets
ax = gp.dotplot(enr_res,figsize=(3,5),
                x='UP_DW',
                x_order = ["UP","DOWN"],
                title="GO_BP",
                cmap = NbDr.reversed(),
                size=3,
                show_ring=True)
ax.set_xlabel("")
plt.show()

ax = gp.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="GO_BP",
                color = ['b','r'])

### GEX dot plots

In [ ]:
# Evaluate collated markers in dictionary (from three papers)

marker_genes_by_cell_type = {
    'Neuroepithelial cells': ('Wnt8b', 'Hmga2', 'Top2a', 'Ube2c', 'Id4'),
    'Radial glial cells': ('Top2a', 'Ube2c', 'Id4','Pax6'),
    'Intermediate progenitor\n cellss': ('Btg2', 'Eomes','Id4', 'Top2a', 'Hes5', 'Pax6', 'Sox2'),
    'Neurons - Immature': ('Nrp1','Neurod6', 'Tubb3', 'Grin2a'),
    'Neurons - Layer II - III ': ('Cux1', 'Cux2',),
    'Neurons - Layer II – IV ': ('Satb2',),
    'Neurons - Layer V – VI ': ('Bcl11b',),
    'Cajal-Retzius cells': ('Reln'),
    'Interneurons': ('Adarb2', 'Bcl11b', 'Gad2'),
    'Oligodendroyte IPCs': ('Sox10', 'Cspg4'),
    'Oligodendrocytes': ('Pdgfra',),
    'Astrocytes': ('Aldh1l1', 'Slc1a3', 'S100b', 'Apoe', 'Gfap', 'Aldoc'),
    'Microglia': ('Tyrobp'),
    'Endothelial': ('Cldn5', 'Igfbp7', 'Rgs5'),
    'Pericyte': ('Col3a1',)

}

In [ ]:
# Make a dotplot of labelled celltypes compared to marker gene canonical cell types (defined by lit)

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=['cell_type'])

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=['cell_type', 'genotype'])


# left: my cell types 
# top: marker gene cell types
# bottom: marker genes defined by lit 

In [ ]:
# Loop through list of age related references 
for x in age_references:
    # Extract labels from data
    labels = data.obs[x]
    
    # Convert labels to integers using pd.factorize
    label_integers, _ = pd.factorize(labels)
    
    # Compute silhouette score
    silhouette = silhouette_score(data.obsm['X_spectral'], label_integers, metric='cosine')
    
    # Print the result
    #print(x + ' ' + str(silhouette))

# label_dibella_2021_E13.5 0.19329716666848698
# label_zhang_2021_E14.5 0.09768439100479863
# label_loo_2019_E14.5 0.12570426820771097
# label_dibella_2021_E14.5 0.0827230839703915
# label_dibella_2021_E17.5 -0.11572978932738838
# label_zhang_2021_E18.5 0.061716785633409314
# label_dibella_2021_E18.5b -0.0054670941269754515
# label_dibella_2021_E18.5a -0.13035388402899664

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=[x], title=x + ' ' + str(silhouette), colorbar_title='Mean gene expression\nin cell-type group')

In [ ]:
# Compare to own score 

silhouette_score(data.obsm['X_spectral'], data.obs["cell_type"], metric='cosine')

### Cellbender effectiveness evaluation

In [ ]:
# Evaluate CellBender with before vs. after plots 

#Before: 
sc.pl.dotplot(data, marker_genes_by_cell_type, groupby=['cell_type'], layer='raw')

# Rows look similar to each other in their expression level 

In [ ]:
# After Cellbender 

sc.pl.dotplot(data, marker_genes_by_cell_type, groupby=['cell_type'])

# reduced expression of off-target genes, more distinct expression per cluster. Each row looks more unique

### Age classifications of marker gene dictionaries

In [ ]:
# UPDATED 

cluster_cell_type_dict_14 = {
    '0': 'Immature neurons',
    '1': 'Progenitor cells',
    '2': 'Immature neurons',
    '3': 'Progenitor cells',
    '4': 'Interneurons',
    '5': 'Immature neurons',
    '6': 'Immature neurons',
    '7': 'Interneurons',
    '8': 'Immature neurons',
    '9': 'Immature neurons',
    '10': 'Progenitor cells',
    '11': 'Immature neurons',
    '12': 'Immature neurons',
    '13': 'Interneurons',
    '14': 'Immature neurons',
    '15': 'Progenitor cells',
    '16': 'Immature neurons',
    '17': 'Pericytes',
    '18': 'Interneurons',
    '19': 'Cajal-Retzius cells',
    '20': 'Neurons',
    '21': 'Endothelial cells',
    '22': 'Microglial cells',
    '23': 'Oligodendrocyte progenitor cells',
}

# Define a dictionary to map cell types to colors
E14_cell_type_colors = {
    'Immature neurons': 'red',
    'Progenitor cells': 'orange',
    'Neurons': 'yellow',
    'Interneurons': 'green',
    'Pericytes': 'purple',
    'Cajal-Retzius cells': 'cyan',
    'Endothelial cells': 'magenta',
    'Microglial cells': 'blue',
    'Astrocytes': 'brown',
    'Oligodendrocyte progenitor cells':'grey'
}

for age_tag in data.obs["age"].unique():
    if age_tag == 14:
        for sample_tag in data.obs["genotype"].unique():
            subset_data = data[data.obs["genotype"] == sample_tag]
            subset_data.obs['cell_type'] = subset_data.obs['leiden'].map(cluster_cell_type_dict_14[age_tag])
            print(subset_data)
            # Map the cell types to colors
            subset_data.uns['E14_cell_type_colors'] = {cell_type: E14_cell_type_colors[cell_type] for cell_type in subset_data.obs['cell_type'].unique()}
            
            sc.pl.umap(subset_data, color=['cell_type'], title=age_tag + ' ' + sample_tag + ' Cell Types', frameon=False, palette = E14_cell_type_colors)
            #snap.pl.umap(subset_data, color=['cell_type'])

In [ ]:
#E18

# UPDATED 

# Define a dictionary to map cell types to leiden clusters
cluster_cell_type_dict_18 = {
        '0': 'Neurons',
        '1': 'Progenitor cells',
        '2': 'Neurons',
        '3': 'Interneurons',
        '4': 'Neurons',
        '5': 'Neurons',
        '6': 'Neurons',
        '7': 'Interneurons',
        '8': 'Neurons',
        '9': 'Neurons',
        '10': 'Neurons',
        '11': 'Interneurons',
        '12': 'Neurons',
        '13': 'Interneurons',
        '14': 'Neurons',
        '15': 'Astrocytes',
        '16': 'Neurons',
        '17': 'Pericytes',
        '18': 'Interneurons',
        '19': 'Cajal-Retzius cells',
        '20': 'Neurons',
        '21': 'Endothelial cells',
        '22': 'Microglial cells',
        '23': 'Oligodendrocytes',
    }


# Define a dictionary to map cell types to colors
E18_cell_type_colors = {
    'Immature neurons': 'red',
    'Progenitor cells': 'orange',
    'Neurons': 'yellow',
    'Interneurons': 'green',
    'Pericytes': 'purple',
    'Cajal-Retzius cells': 'cyan',
    'Endothelial cells': 'magenta',
    'Microglial cells': 'blue',
    'Astrocytes': 'brown',
    'Oligodendrocytes': 'grey'
}


for age_tag in data.obs["age"].unique():
    if age_tag == 18:  # Only proceed if age_tag is 18
        for sample_tag in data.obs["genotype"].unique():
            subset_data = data[data.obs["genotype"] == sample_tag]
            subset_data.obs['cell_type'] = subset_data.obs['leiden'].map(cluster_cell_type_dict_18[age_tag])
            # Map the cell types to colors
            subset_data.uns['E18_cell_type_colors'] = {cell_type: cell_type_colors[cell_type] for cell_type in subset_data.obs['cell_type'].unique()}
            sc.pl.umap(subset_data, color=['age','condition'], title=str(age_tag) + ' ' + sample_tag + ' Cell Types', frameon=False, palette=E18_cell_type_colors)

# for age_tag in data.obs["age"].unique():
#     for sample_tag in data.obs["genotype"].unique():
#         subset_data = data[data.obs["genotype"] == sample_tag]
#         subset_data.obs['cell_type'] = subset_data.obs['leiden'].map(cluster_cell_type_dict_18[age_tag])
#         # Map the cell types to colors
#         subset_data.uns['cell_type_colors'] = {cell_type: cell_type_colors[cell_type] for cell_type in subset_data.obs['cell_type'].unique()}
#         sc.pl.umap(subset_data, color=['age','condition'], title=age_tag + ' ' + sample_tag + ' Cell Types', frameon=False, palette = E18_cell_type_colors)


## Marker gene plots

In [ ]:
# Define your marker groups
marker_groups = {
    'cluster 1': ['ldn5', 'Flt1', 'Ptprm', 'Ptprg', 'Sptbn1', 'Tjp1'],
    'cluster 2': ['Sox6', 'Npas3', 'Nfia', 'Qk', 'Mir9-3hg', 'Mir99ahg'],
    'cluster 3': ['Ank3', 'Meg3', 'Snhg14', 'Fam155a', 'Erc2', 'Rims2'],
    'cluster 4': ['Nfib', 'Sox5', 'Rbfox1', 'Ptprd', 'Nav3', 'Cdh13'],
    'cluster 5': ['Sorbs2', 'Igfbpl1', 'Tcf4', 'Cacna2d1', 'Unc5d', 'Epha3'],
    'cluster 6': ['Ebf1', 'Nrxn3', 'Bcl11b', 'Meg3', 'Robo1', 'Nrxn1'],
    'cluster 7': ['Gad2', 'Nrxn3', 'Dlx6os1', 'Sox2ot', 'Gm38505', 'Gad1'],
    'cluster 8': ['Erbb4', 'Nrxn3', 'Erbb4', 'Dlgap1', 'Xkr4', 'Gria4'],
    'cluster 9': ['Gli3', 'Sox6', 'Creb5', 'Etl4', 'Cdon', 'Mir99ahg'],
    'cluster 10': ['Dlc1', 'Prkg1', 'Ebf1', 'Itga1', 'Cald1', 'Plcl1'],
    'cluster 11': ['Top2a', 'Mki67', 'Zbtb20', 'Diaph3', 'H2afz', 'Smc4'],
    'cluster 12': ['Ptprd', 'Tafa1', 'Celf2', 'Adgrl3', 'Tafa2', 'Grin2b'],
    'cluster 13': ['Rora', 'Fbxl7', 'Cped1', 'Sdk1', 'Colec12', 'Ror1'],
    'cluster 14': ['Magi1', 'Setbp1', 'Dleu2', 'Mir99ahg', 'Nrg1', 'Sox6'],
    'cluster 15': ['Adgrv1', 'Msi2', 'Zbtb20', 'Npas3', 'Qk', 'Mir99ahg'],
    'cluster 16': ['Meis2', 'Nrxn3', 'Meis2', 'Ptprm', 'Dlg2', 'Bcl11b'],
    'cluster 17': ['Nrg3', 'Ptprd', 'Mdga2', 'Nrxn1', 'Arpp21', 'Grin2b'],
    'cluster 18': ['Reln', 'Dach1', 'Msi2', 'Cep112', 'Fat3', 'Maml3'],
    'cluster 19': ['Slit3', 'Msra', 'Nrp1', 'Ank3', 'Nrxn1', 'Ntrk3'],
    'cluster 20': ['Satb2', 'Dlg2', 'Arpp21', 'Frmd4a', 'Celf2', 'Ptprd'],
    'cluster 21': ['Mbnl1', 'Lgmn', 'Apoe', 'Slc9a9', 'C1qb', 'Maf'],
    'cluster 22': ['Nxph1', 'Nrxn3', 'Erbb4', 'Nxph1', 'Gria4', 'Sox2ot'],
    'cluster 23': ['Ptn', 'Ptprz1', 'Tnc', 'Qk', 'Slc1a3', 'Zbtb20'],
    'cluster 24': ['Unc5d', 'Meis2', 'Igfbpl1', 'Nrg1', 'Trim2', 'Mir99ahg'],
    'cluster 25': ['Frmd4a', 'Gria2', 'Satb2', 'Rbfox1', 'Ttc28', 'Frmd4b'],
    'cluster 26': ['Nav3', 'Ptprd', 'Nrg3', 'Celf2', 'Arpp21', 'Lrfn5']
}

# Convert marker groups dictionary to a DataFrame
cluster_data = []
for cluster, markers in marker_groups.items():
    marker_string = ', '.join(markers)
    cluster_data.append([cluster, marker_string])

# Create DataFrame
cluster_df = pd.DataFrame(cluster_data, columns=['Cluster', 'Markers'])

# Display the DataFrame
print(cluster_df)


In [ ]:
# Generating marker gene plots using your canonical markers and grouping by each of the label sets - and if you can group by multiple columns (not sure if this is possible) you should group by age and maybe con Vs hom too

# Ruan reference 

Ruan_marker_genes_by_cell_type = {
    'NEC': ['Wnt8b', 'Hmga2', 'Top2a', 'Ube2c', 'Id4'],
    'RGC': ['Top2a', 'Ube2c', 'Id4', 'Pou3f2', 'Zbtb20'],
    'IPC': ['Top2a', 'Ube2c', 'Id4', 'Eomes', 'Neurog2', 'Neurod1'],
    'Immature neurons': ['Neurod1'],
    'Layer II – IV neurons': ['Satb2'],
    'Layer V – VI neurons': ['Bcl11b'],
    'Hippocampus': ['Tle4', 'Bcl11b'],
    'Layer I neurons': ['Neurod1', 'Bcl11b', 'Crym'],
    'Interneurons': ['Bcl11b', 'Dlx1'],
    'OPC': ['Top2a', 'Id4', 'Olig1'],
    'Pia': ['Top2a', 'Hmga2', 'Ube2c', 'Col3a1'],
    'Pericyte': ['Top2a', 'Ube2c', 'Col3a1', 'Rgs5'],
    'Microglia': ['Top2a', 'Ube2c', 'Tle4', 'Tyrobp'],
    'Endothelial': ['Cldn5']
}

# Dibella reference 
Dibella_marker_genes_by_cell_type = {
    'Apical progenitors': ['Sox2'],
    'Intermediate progenitors': ['Eomes'],
    'Migrating neurons': ['Nrp1'],
    'Neurons': ['Tubb3'],
    'Callosal PN': ['Satb2'],
    'Layer II & III callosal PN': ['Cux2'],
    'Subcerebral PN': ['Bcl11b'],
    'Corticothalamic PN': ['Tle4'],
    'Cajal-Retzius cells': ['Reln'],
    'Interneurons': ['Dlx2'],
    'Oligodendrocytes': ['Pdgfra'],
    'Astrocytes': ['Apoe'],
    'Endothelial cells': ['Cldn5'],
    'Microglia': ['Aif1'],
    'Red blood cells': ['Car2'],
    'VLMC': ['Col1a1']
}

# Loo reference 
Loo_marker_genes_by_cell_type = ['Hes1', 'Sox2', 'Neurog2', 'Eomes', 'Btg2', 'Igfbpl1', 'Pcp4', 'Nrp1', 'Neurod2', 'Neurod1', 'Unc5d', 'Sema3c', 'Tbr1', 'Bcl11b', 'Tubb3', 'Sox5', 'Fezf2', 'Reln', 'Calb2', 'Emx2', 'Lhx5', 'Tle4', 'Tshz2', 'Ldb2', 'Etv1', 'Nr4a2', 'Rorb', 'Lpl', 'Ptn', 'Satb2', 'Cux1', 'Dlx1', 'Gad2', 'Lhx6', 'Cux2', 'Apoe', 'Aldoc', 'Slc1a3', 'Olig1', 'Pdgfra', 'Olig2']


# Ordered list of paper marker genes + canonical markers 

#paper_list = ['Neurod6', 'Satb2', 'Bcl11b', 'Gad2', 'Lhx1', 'Reln', 'Trp73', 'Lhx5', 'Mki67', 'Bmp2', 'Ascl1', 'Olig2', 'Lhx1', 'Wnt7b', 'Eomes','Foxp2','Hoxb3', 'Hap1', 'Nkx2-1', 'Sez6l2', 'Tbx20', 'Nfib', 'Cntn1','Dcx']
Loo_gene_names = ['Gad2', 'Reln', 'Bcl11b', 'Satb2', 'Sox5', 'Hbb-bs','Dcx', 'Fezf2', 'Neurod6', 'Gad2', 'Bcl11b', 'Satb2', 'Foxp2', 'Nrgn', 'Inhba', 'Top2a', 'Mki67','Lhx1', 'Reln', 'Trp73', 'Lhx5', 'Nxph1', 'Maf', 'Erbb4', 'Sst', 'Npy', 'Aqp4', 'Aldoc', 'Aldh1l1', 'Slc1a3', 'Slc6a11', 'Hes1', 'Hes5', 'Tbr1', 'Eomes',  'Bmp2', 'Pax6', 'Nfib', 'Cntn1', 'Ednrb','Trem2', 'Mpeg1', 'P2ry12', 'Epas1', 'Emcn', 'Olig1', 'Olig2']
# not present: 'Pvrl3', 'Int1', 'Int2'Int3Int4Pou3fl

Loo_E18_genes = ['Neurod6', 'Nes', 'Slc17a6', 'Vim', 'Slc17a6', 'Bcl11b', 'Fezf2', 'Tbr1', 'Reln', 'Trp73', 'Lhx1', 'Lhx5', 'Satb2', 'Dok5', 'Inhba', 'Nrgn', 'Lpl', 'Nxph3', 'Hs3st4', 'Tfap2d', 'Rwdd3', 'Eomes', 'Sema3c', 'Neurod1', 'Unc5d', 'Calb2', 'Crym', 'Dlx1', 'Dlx2', 'Gad1', 'Gad2', 'Lhx6', 'Nxph1', 'Lhx8', 'Resp18', 'Htr3a', 'Adarb2', 'Tiam2', 'Isl1', 'Syt6', 'Gpr88', 'Rxrg', 'Mki67', 'Top2a', 'Spag5', 'Ccna2', 'Aqp4', 'Aldh1l1', 'Hes1', 'Hes5', 'Entpd2', 'Pla2g3', 'Olig2', 'Pdgfra', 'Dnah12', 'Iqca', 'Ak7', 'Folr1', 'Kcne2', 'Otx2', 'Trem2', 'Mpeg1', 'Igfbp7', 'Epas1', 'Ushbp1', 'Col3a1', 'Abcc9', 'Kcnj8']
#not present: Pcam1, Sgol2
Loo_E14_genes = ['Neurod6', 'Nes', 'Slc17a6', 'Vim', 'Slc17a6', 'Bcl11b', 'Fezf2', 'Tbr1', 'Reln', 'Trp73', 'Lhx1', 'Lhx5', 'Crym', 'Ndrg1', 'Mc4r', 'Nfe2l3', 'Nxph3', 'Sla', 'Sybu', 'Tfap2d', 'Rwdd3', 'Ppp1r14c', 'Satb2', 'Cux2', 'Eomes', 'Tiam2', 'Pdzrn3', 'Nhlh1', 'Unc5d', 'Calb2', 'Dlx1', 'Dlx2', 'Top2a', 'Mki67', 'Btbd17', 'Spag5', 'Ednrb', 'Tk1', 'Tcf19', 'Pkmyt1', 'Arhgef39', 'Rspo1', 'Dlx1', 'Dlx2', 'Gad1', 'Lhx6', 'Nxph1', 'Gad2', 'Dkk3', 'Pnoc', 'Isl1', 'Syt6', 'Gpr88', 'Rxrg', 'Shox2', 'Syt13', 'Ak7', 'Folr1', 'Foxj1', 'Kcne2', 'Otx2', 'Mpeg1', 'Igfbp7', 'Epas1', 'Pecam1', 'Col3a1']

combined_marker_gene_list = []

### Gene Expression Plots

In [ ]:
# Evaluate collated markers in dictionary (from three papers


marker_genes_by_cell_type = {
    'Neural stem cells': ('Wnt8b', 'Hmga2', 'Top2a', 'Ube2c', 'Id4'),
    'Radial glial cells': ('Top2a', 'Ube2c', 'Id4', 'Pax6'),
    'Intermediate progenitors': ('Top2a', 'Ube2c', 'Id4', 'Eomes', 'Neurog2', 'Neurod1'),
    'Apical progenitors': ('Sox2', 'Pax6', 'Hes5'),
    'Migrating neurons': ('Nrp1',),
    'Neurons': ('Tubb3',),
    'Layer I neurons': ('Bcl11b', 'Crym'),
    'Layer II & III callosal PN': ('Cux2',),
    'Layer II – IV neurons': ('Satb2',),
    'Layer V – VI neurons': ('Bcl11b',),
    'Excitatory neurons': ('Neurod6',),
    'Interneurons': ('Bcl11b', 'Dlx1', 'Sst', 'Gad2', 'Adarb2', 'Isl1'),
    'Subcerebral PN': ('Bcl11b',),
    'Corticothalamic PN': ('Tle4',),
    'Hippocampus': ('Tle4', 'Bcl11b'),
    'Proliferating cells': ('Mki67',),
    'Oligodendroyte progenitor cells': ('Top2a', 'Id4', 'Olig1'),
    'Oligodendrocytes': ('Pdgfra',),
    'Astrocytes': ('Apoe', 'Hes5', 'Aldh1l1', 'Slc1a3'),
    'Microglia': ('Top2a', 'Ube2c', 'Tle4', 'Tyrobp'),
    'Endothelial': ('Cldn5', 'Igfbp7'),
    'Pia': ('Top2a', 'Hmga2', 'Ube2c', 'Col3a1'),
    'Pericyte': ('Top2a', 'Ube2c', 'Col3a1', 'Rgs5'),
    'Cajal-Retzius cells': ('Reln', 'Wnt8b'),
    'Red blood cells': ('Car2','Hemgn')
}

# Make a dotplot of labelled celltypes compared to marker gene canonical cell types (defined by lit)

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=['cell_type'], dendrogram=True)

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=['cell_type', 'age'])


# left: my cell types 
# top: marker gene cell types
# bottom: marker genes defined by lit 

In [ ]:
# Make a dotplot of labelled celltypes compared to marker gene canonical cell types (defined by lit)

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=['cell_type'], dendrogram=True)

sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=['cell_type', 'age'])


# left: my cell types 
# top: marker gene cell types
# bottom: marker genes defined by lit 

In [ ]:
# Using merged list of canonical marker genes from the three papers, look at the label transfer labelling
# Per reference, look at spectral leiden cluster labelling. Compare with UMAPS. 
for ref in age_references:
    # Dynamically create the list of column names to group by
    groupby_cols = [ref, "age", "genotype"]
    
    # Pass the list of column names directly to the `groupby` parameter
    sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=[ref, 'leiden'], title=ref)


In [ ]:
# Using merged list of canonical marker genes from the three papers, look at the label transfer labelling
for ref in age_references:
    # Dynamically create the list of column names to group by
    groupby_cols = [ref, "age", "genotype"]
    
    # Pass the list of column names directly to the `groupby` parameter
    sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=[ref])


In [ ]:
# references = ['label_loo_2019_E14.5', 'label_loo_2019_P0', 'label_dibella_2021_E13.5', 'label_zhang_2021_E10.5', 'label_dibella_2021_E14.5', 'label_dibella_2021_E15.5', 'label_dibella_2021_E10.5', 'label_zhang_2021_E16.5', 'label_zhang_2021_E18.5', 'label_zhang_2021_E12.5', 'label_zhang_2021_E14.5', 'label_dibella_2021_E11.5', 'label_dibella_2021_E12.5', 'label_zhang_2021_E15.5', 'label_dibella_2021_E17.5', 'label_dibella_2021_E16.5', 'label_dibella_2021_E18.5b', 'label_dibella_2021_E18.5a']

for x in references: 
    combined_column = (
        data.obs['aAge'],
        data.obs['condition'],
        x
    )
    # Create a new column with the combined information
    data.obs['label_age_condition'] = combined_column

    # Plot marker gene expression grouped by reference
    sc.pl.dotplot(data, marker_genes_by_cell_type, groupby="label_age_condition")


#for every leiden cluster, look at where it is on the UMAP. What cell types are in the same position on UMAP. Look at gene exp of cluster compared to reference. 
#leiden cluster goes across multiple cell types (per references), go to marker gene expression and decide which to trust. 
# save cluster annotation into a h5ad file leiden clustering. add save step to end of workflow to post=processed h5ad. final step, once have done leiden clustering. save again once assigned clusters to labels. save what cells below to cluster 1 and 2. 

In [ ]:
for x in age_references: 
    sc.pl.dotplot(data, var_names=marker_genes_by_cell_type, groupby=[x,'leiden'], title=x)

In [ ]:
# Create a column with merged age and condition to groupby these two together 
for x in age_references: 
    combined_column = data.obs['age'].astype(str) + '_' + data.obs['condition'].astype(str) + '_' + [x]

# Create a new column with the combined information
    data.obs['label_age_condition'] = combined_column
#Plot marker gene expression grouped by reference
    sc.pl.dotplot(data, marker_genes_by_cell_type, groupby="label_age_condition")

### Cellbender evaluation

In [ ]:
# Evaluate CellBender with before vs. after plots 

#Before: 
sc.pl.dotplot(data, marker_genes_by_cell_type, groupby=['cell_type'], layer='raw')

# Rows look similar to each other in their expression level 

In [ ]:
# Evaluate CellBender with before vs. after plots 

#Before: 
sc.pl.dotplot(data, marker_genes_by_cell_type, groupby=['cell_type'], layer='raw')

# Rows look similar to each other in their expression level 

# After Cellbender 

sc.pl.dotplot(data, marker_genes_by_cell_type, groupby=['cell_type'])

# reduced expression of off-target genes, more distinct expression per cluster. Each row looks more unique

### GEX Dotplots

In [ ]:
# make gene expression dot plots using the markers from literature but grouping by transferred labels (and maybe age if you're able to group by two columns), which will give you an idea of how consistent the transferred labels are

In [ ]:
sc.tl.rank_genes_groups(data, groupby="label_loo_2019_E14.5")

In [ ]:
sc.pl.umap(data, color=["sample", "label_loo_2019_E14.5"])

In [ ]:
interesting_genes = ["Nsd1", "Eomes", "Pax6", "Ki67", "pHH3", "CC3"]

marker_set = set(np.array([x for x in data.uns[f"{clustering_key}_markers"].values()]).flatten()) - set(interesting_genes)

In [ ]:
sc.pl.dotplot(data, [*interesting_genes, *marker_set], groupby="source")

In [ ]:
rank_data = data.uns["rank_genes_groups"]

df = pd.concat(
    [
        pd.DataFrame({
            "cluster": cluster,
            "gene": rank_data["names"][cluster],
            "pvals_adj": rank_data["pvals_adj"][cluster],
            "logfc": rank_data["logfoldchanges"][cluster]
        })
        for cluster in rank_data["names"].dtype.names
    ],
    ignore_index=True
)

print("\n".join(df[(df.cluster == "1") & (df.pvals_adj < 0.05) & (df.logfc > 0)].sort_values("logfc").tail(10)["gene"]))

In [ ]:
# E14

# Define a dictionary to map cell types to leiden clusters
cluster_cell_type_dict_14 = {
    '0': 'Immature neurons',
    '1': 'Progenitor cells',
    '2': 'Neurons',
    '3': 'Progenitor cells',
    '4': 'Interneurons',
    '5': 'Neurons',
    '6': 'Neurons',
    '7': 'Interneurons',
    '8': 'Interneurons',
    '9': 'Progenitor cells',
    '10': 'Progenitor cells',
    '11': 'Neurons',
    '12': 'Neurons',
    '13': 'Interneurons',
    '14': 'Neurons',
    '15': 'Progenitor cells',
    '16': 'Neurons',
    '17': 'Pericytes',
    '18': 'Interneurons',
    '19': 'Cajal-Retzius cells',
    '20': 'Neurons',
    '21': 'Endothelial cells',
    '22': 'Microglial cells',
    '23': 'Progenitor cells',
}

# Define a dictionary to map cell types to colors
E14_cell_type_colors = {
    'Immature neurons': 'red',
    'Progenitor cells': 'orange',
    'Neurons': 'blue',
    'Interneurons': 'green',
    'Pericytes': 'purple',
    'Cajal-Retzius cells': 'cyan',
    'Endothelial cells': 'magenta',
    'Microglial cells': 'yellow',
    'Astrocytes': 'brown',
}

for age_tag in data.obs["age"].unique():
    for sample_tag in data.obs["condition"].unique():
        subset_data = data[data.obs["condition"] == sample_tag]
        subset_data.obs['cell_type'] = subset_data.obs['leiden'].map(cluster_cell_type_dict_combined[age_tag])
        # Map the cell types to colors
        subset_data.uns['E14_cell_type_colors'] = {cell_type: E14_cell_type_colors[cell_type] for cell_type in subset_data.obs['cell_type'].unique()}
        sc.pl.umap(subset_data, color=['cell_type'], title=age_tag + ' ' + sample_tag + ' Cell Types', frameon=False, palette = E14_cell_type_colors)
        #snap.pl.umap(subset_data, color=['cell_type'])

### Manual Annotation

In [ ]:
markers = {
    "AQP4": ["Astrocytes"],
    "GJA1": ["Astrocytes"],
    "FOXJ1": ["Astrocytes"],
    "Slc8a1": ["Astrocytes"],
    "Slc1a2": ["Astrocytes"],
    "GFAP": ["Astrocytes"],
    "Aqp4": ["Astrocytes"],
    "Aldh1l1": ["Astrocytes"],
    "Olig1": ["Astrocytes","Oligodendrocytes"],
    "Olig2": ["Astrocytes","Oligodendrocytes"],
    "Qk": ["Oligodendrocyte progenitor cells", "Oligodendrocytes"],
    
    "Pax6": ["Radial glial cells"],
    "FABP7": ["Radial glial cells"],
    "VIM": ["Radial glial cells"],
    "NES": ["Radial glial cells"],
    "Sox2": ["Radial glial cells"],
    "Hes1": ["Radial glial cells"],
    "Hes5": ["Radial glial cells"],
    "Ednrb": ["Radial glial cells"],
    "Sptbn1": ["Radial glial cells"],
    "A2B5": ["Glial progenitor cells"],
    
    "Neurod6": ["Excitatory neurons"],
    "Grin2b": ["Excitatory neurons"],
    "Slc2a1": ["Excitatory neurons"],
    "Gria4": ["Excitatory neurons"],
    "Slc1a3": ["Excitatory neurons"],
    "Dlg2": ["Excitatory neurons"],
    "Slc6a1": ["Excitatory neurons"],
    "Bcl11b": ["Excitatory neurons", "Neural progenitor cells"],
    "Erbb4": ["Excitatory neurons", "GABAergic cells", "Interneurons"],
    "Nrxn1": ["Excitatory neurons", "Inhibitory interneurons"],
    "Celf2": ["Excitatory neurons", "Inhibitory neurons"],
    "Rims2": ["Excitatory neurons", "Inhibitory neurons"],
    "Ank3": ["Excitatory neurons", "Inhibitory neurons"],
    "Dlx1": ["Excitatory neurons", "Inhibitory neurons"],
    "Erc2": ["Excitatory neurons", "Inhibitory neurons"],
    "Nrxn3": ["Excitatory neurons", "Inhibitory neurons"],
    "Arpp21": ["Excitatory neurons", "Inhibitory neurons"],
    
    "Gad1": ["Inhibitory neurons"],
    "Gad2": ["Inhibitory neurons"],
    "Calb2": ["Inhibitory neurons"],
    "Isl1": ["Inhibitory neurons"],
    "Gpr88": ["Inhibitory neurons"],

    "Tcf7l2": ["Thalamus"],
    "Syt13": ["Thalamus"],
    
    "Trem2": ["Microglial cells"],
    "CD163": ["Microglial cells"],
    "TYROBP": ["Microglial cells"],
    "DOCK8": ["Microglial cells"],
    "GZMB": ["Microglial cells"],
    "FCN1": ["Microglial cells"],
    "TMEM119": ["Microglial cells"],
    "Aif1": ["Microglial cells"],
    "Iba-1": ["Microglial cells"],
    "Cx3cr1": ["Microglial cells"],
    "Cd68": ["Microglial cells"],
    "C1qb": ["Microglial cells"],
    "Apoe": ["Microglial cells"],
   
    "Reln": ["Cajal-Retzius cells"],
    "Trp73": ["Cajal-Retzius cells"],
    "Lhx1": ["Cajal-Retzius cells"],
    "Lhx5": ["Cajal-Retzius cells"],
    "Nrg1": ["Cajal-Retzius cells"],
    "Fat3": ["Cajal-Retzius cells"],
    "Rspo1-3": ["Cajal-Retzius cells"],
    "Dkk3": ["Cajal-Retzius cells"],
    "Vim": ["Cajal-Retzius cells"],
    
    "Int1": ["Interneurons"],
    "Int2": ["Interneurons"],
    "Int3": ["Interneurons"],
    "Int4": ["Interneurons"],
    "Lhx6": ["Interneurons"],
    "Prox1": ["Interneurons"],
    "Sp9": ["Interneurons"],
    "Tiam2": ["Interneurons"],
    "Sox6": ["Interneurons"],
    "Dlx5": ["Interneurons"],

    "Cldn5": ["Endothelial cells"],
    "Flt1": ["Endothelial cells"],
    "Ramp2": ["Endothelial cells"],
    "Fli1": ["Endothelial cells"],
    "Ptprm": ["Endothelial cells"],
    "Igfbp7": ["Endothelial cells"],
    
    "Zfp521": ["Neural stem cells"], 
    "Mki67": ["Neural stem cells"],
    "Top2a": ["Neural stem cells"],
    "Kif15": ["Neural stem cells"],
    "Rmst": [ "Neural stem cells"],
    "Nestin": ["Neural stem cells"],
    "Sox2": ["Neural stem cells"],
    "Sox9": ["Neural stem cells"],
    "Sema3c": ["Neural stem cells"],
    "Ptprd": ["Neural stem cells"],
               
    "Bmp2": ["Neural progenitor cells"],
    "Eomes": ["Neural progenitor cells"],
    "Phip": ["Neural progenitor cells"],
    "Ctip2": ["Neural progenitor cells"],
    "Lmx1a": ["Neural progenitor cells"],
    "Auts2": ["Neural progenitor cells"], # transcriptional target of Tbr1 in developing neocortex
    "Ctnna2": ["Neural progenitor cells"],
    "Chd7": ["Neural progenitor cells"],
    "Cbfa2t2": ["Neural progenitor cells"],
    "Tcf12": ["Neural progenitor cells"],
    "Pbx3": ["Neural progenitor cells"],
    "Meis2": ["Neural progenitor cells"],
    "Cenpf": ["Neural progenitor cells"],
    "Dach1": ["Neural progenitor cells"],
    "Sox5": ["Neural progenitor cells"],
    "Gli3": ["Neural progenitor cells"],
    "Tcf4": ["Neural progenitor cells"],
    "Nfia": ["Neural progenitor cells"],
    "Rbfox1": ["Neural progenitor cells"],
    "Tnc": ["Neural progenitor cells"],
    "Maml3": ["Neural progenitor cells", "Radial glial cells"],
    "Cep112": ["Neural progenitor cells", "Radial glial cells"],
    "Msi2": ["Neural stem cells", "Neural progenitor cells"],
    "Wdr17": ["Neural progenitor cells", "Radial glial cells"],
    "Ptn": ["Neural progenitor cells", "Migrating neurons", "Differentiating neurons"],
    "Ebf1": ["Neural progenitor cells", "Differentiating neurons"],
    "Rarb": ["Neural progenitor cells", "Differentiating neurons", "Glial cells"],
    "Zbtb20": ["Neural progenitor cells", "Differentiating neurons", "Glial cells"],
    "Etl4": ["Neural progenitor cells", "Differentiating neurons"],
    "Cdon": ["Neural progenitor cells", "Migrating neurons", "Differentiating neurons"],
    "Fam155a": ["Neural progenitor cells", "Differentiating neurons"],
    "Nrg3": ["Neural progenitor cells", "Migrating neurons", "Neural stem cells"],
    "Tbr1": ["Intermediate progenitor cells"],
    
    "Mef2c": ["Neurons"],
    "Plxna4": ["Neurons"],
    "Tuba1a": ["Neurons"],
    "Ccnd2": ["Neurons"],
    "Rnf220": ["Neurons"],
    "Thsd7a": ["Neurons"],
    "Ntng1": ["Neurons"],
    "Zfpm2": ["Neurons"],
    "Frmd4b": ["Neurons"],
    "Mllt3": ["Neurons"],
    "Slc17a6":["Neurons"],
    "Satb2": ["Neurons"],
    "Pou3fl": ["Neurons"],
    "Nrgn": ["Neurons"],
    "RBFOX3": ["Neurons"],
    "Nav3": ["Neurons"],
    "Kcnq3": ["Neurons"],
    "Map2": ["Neurons"],
    "Tafa2": ["Neurons"],
    "Tuj1": ["Neurons"],
    "Inhba": ["Neurons"],
    "Ascl1": ["Neurons"],
    "Pvrl3": ["Neurons"],
    "Elavl2": ["Neurons"],
    "Adgrv1": ["Neurons"],
    "Grip1": ["Neurons"],
    "Aff3": ["Neurons"],
    "Pam": ["Neurons"],
    "Ctnna2": ["Neurons","Neural progenitor cells", "Radial glial cells"],
    "Adgrl3": ["Neurons", "Astrocytes", "Oligodendrocytes"],
    "Tafa1": ["Neurons", "Glial cells"],
    "Dner": ["Neural progenitor cells", "Radial glial cells", "Neurons"],
    "Enox1": ["Neurons", "Glial cells"],
    "Foxp1": ["Neurons", "Astrocytes", "Oligodendrocytes"],
    "Robo2": ["Neurons", "Glial cells"],
    "Lsamp": ["Neurons", "Radial glial cells"],
    "Epha3": ["Neurons", "Glial cells"],
    "Arhgap6": ["Neurons", "Glial cells"],
    "Mdk": ["Neurons", "Glial cells"],
    "Diaph3": ["Neurons", "Glial cells"],
    "Frmd4a": ["Neurons", "Glial cells"],
    "Ptprk": ["Neurons", "Glial cells"],
    "Mdga2": ["Neurons", "Glial cells"],
    "Ptprg": ["Neurons", "Glial cells"],
    "H2afz": ["Neurons", "Glial cells"],
    "Hmgb2": ["Neurons", "Glial cells"],
    "Ccser1": ["Neurons", "Glial cells"],
    "Epha5": ["Neurons", "Glial cells"],
    "Rora": ["Neurons", "Glial cells"],
    "Fbxl7": ["Neurons", "Glial cells"],
    "Cped1": ["Neurons", "Glial cells"],
    "Opcml": ["Neurons", "Glial cells"],
    "Sdk1": ["Neurons", "Glial cells"],
    "Colec12": ["Neurons", "Glial cells"],
    "Ror1": ["Neurons", "Glial cells"],
    "Lhfpl3": ["Neurons", "Glial cells"],
    "Maml2": ["Neurons", "Glial cells"],
    "Npas3": ["Neurons", "Glial cells"],
    "Dlc1": ["Neurons", "Glial cells"],
    "Prkg1": ["Neurons", "Glial cells"],
    "Itga1": ["Neurons", "Glial cells"],
    "Cald1": ["Neurons", "Glial cells"],  # cell motility
    "Plcl1": ["Neurons", "Glial cells"],
    "Mbnl1": ["Neurons", "Glial cells"],
    "Lgmn": ["Neurons", "Glial cells"],
    "Slc9a9": ["Neurons", "Glial cells"],
    "Maf": ["Neurons", "Glial cells"],  # differentiation
    "Igfbpl1": ["Neurons", "Glial cells"],
    "Ccser1": ["Neurons", "Glial cells"],
    "Creb5": ["Neurons", "Glial cells"],
    "Plcb1": ["Neurons", "Glial cells"],
    "Cacna2d1": ["Neurons", "Glial cells"],
    "Nxph1": ["Neurons", "Glial cells"],
    "Dlgap2": ["Neurons"],  # postsynaptic densities
    "Rbms3": ["Neurons", "Glial cells"],
    "Nfib": ["Neurons", "Glial cells"],  # differentiation and migration
    "Adarb2": ["Neurons", "Glial cells"],
    "Dlx6os1": ["Neural progenitor cells", "Differentiating neurons"],
    "Celf4": ["Neurons", "Glial cells"],
    "Setbp1": ["Neural progenitor cells", "Differentiating neurons"],
    "Magi1": ["Neurons", "Glial cells"],
    "Msra": ["Neurons", "Glial cells"],
    "Trim2": ["Neurons", "Glial cells"],
    "Ptprz1": ["Neurons", "Glial cells"],
    "Nrp1": ["Neurons", "Glial cells", "Endothelial cells"],

    "Nsd1": ["Gene of interest"],
    "Hbb-bs": ["Red blood cells"],
    "Actb": ["Housekeeping gene"],
    "Gadph": ["Housekeeping gene"],

    "Ubb": ["Dying cells"],
    "Cmss1": ["Dying cells"],  # apoptosis, ubiquitination, and cell death
    "Cst3": ["Dying cells"],
    "Hspa8": ["Dying cells"],
    
}